In [7]:
# Data Preparation (CRISP-DM Step)


# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib
import os

# Ensure processed folder exists


# Load raw dataset

df = pd.read_csv("../data/raw/creditcard.csv")
print("Raw shape:", df.shape)
print(df.head())


# Separate features and target
X = df.drop("Class", axis=1).values
y = df["Class"].values
print("Features shape:", X.shape)
print("Target distribution:", np.bincount(y))

# Scale features

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape:", X_train.shape, y_train.shape)
print("Test shape:", X_test.shape, y_test.shape)
print("Fraud ratio in train:", np.mean(y_train))
print("Fraud ratio in test:", np.mean(y_test))

# Handle class imbalance (SMOTE)
print("\nBefore SMOTE:", np.bincount(y_train))

sm = SMOTE(random_state=42, sampling_strategy=1.0)  # fully balance classes
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print("After SMOTE:", np.bincount(y_train_res))


# Save processed datasets

np.save("../data/processed/X_train.npy", X_train_res)
np.save("../data/processed/y_train.npy", y_train_res)
np.save("../data/processed/y_test.npy", y_test)

joblib.dump(scaler, "../data/processed/scaler.pkl")

print("\n✅ Data preparation complete. Processed files saved in data/processed/")


Raw shape: (284807, 31)
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        